In [ ]:
#Required for the BT model
# !pip install nlpaug

In [ ]:
import numpy as np
import pandas as pd
import nlpaug.augmenter.word as naw
import re
import timeit
#import nltk
#from nltk.tokenize import RegexpTokenizer

In [ ]:
#Read csv files
tweet_input = pd.read_csv('C:/Users/saath/Documents/DAEN 690/datasets/train1.csv')
comment_input = pd.read_csv('C:/Users/saath/Documents/DAEN 690/datasets/train2.csv')

In [ ]:
#Drop unnecessary columns
tweet_input_final = tweet_input.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'],axis=1)
comment_input_final = comment_input.drop(['id', 'toxic', 'severe_toxic', 'threat','insult', 'identity_hate'],axis=1)

# Reordering the columns in the tweet_input
tweet_input_final = tweet_input_final.reindex(['tweet','class'], axis=1)

#Renaming the columns into a common name
comment_input_final.rename(columns={'comment_text': 'text_data', 'obscene': 'profane_class'}, inplace=True)
tweet_input_final.rename(columns={'tweet': 'text_data', 'class': 'profane_class'}, inplace=True)

#Considering only the profane and non profane classes
tweet_input_final = tweet_input_final[tweet_input_final.profane_class != 0]
tweet_input_final['profane_class'] = tweet_input_final['profane_class'].replace([2],0)
# tweet_input_final.head(10)

In [ ]:
#Clean data by removing punctuations, numeric data and http links
# comment_input_final['text_data'] = comment_input_final.text_data.str.replace(r'[^a-zA-Z\s]+@\w+:|@\w+|&\w+|[^a-zA-Z\s]+|RT|http.+?', '')
# comment_input_final['text_data'] = comment_input_final.text_data.str.replace(r'\n',' ')
# comment_input_final.head(10)

In [ ]:
tweet_input_final['text_data'] = tweet_input_final.text_data.str.replace(r'[^a-zA-Z\s]+@\w+:|@\w+|&\w+|[^a-zA-Z\s]+|RT|http.+?', '')
tweet_input_final['text_data'] = tweet_input_final.text_data.str.replace(r'\n',' ')

In [ ]:
back_translation_aug = naw.BackTranslationAug(from_model_name='Helsinki-NLP/opus-mt-en-es', 
                                              to_model_name='Helsinki-NLP/opus-mt-es-en')

In [ ]:
start = timeit.default_timer()
actual_data = tweet_input_final[tweet_input_final.profane_class == 1]['text_data'].to_list()
transformed_data = back_translation_aug.augment(actual_data)
stop = timeit.default_timer()

In [ ]:
# start = timeit.default_timer()
# actual_data = comment_input_final[comment_input_final.profane_class == 1]['text_data'].to_list()
# transformed_data_comment = back_translation_aug.augment(actual_data)
# stop = timeit.default_timer()

In [ ]:
print('Time: ', stop - start)  

In [ ]:
transformed_tweet_data = pd.DataFrame()
# transformed_comment_data = pd.DataFrame()

In [ ]:
se = pd.Series(transformed_data)
transformed_tweet_data['text_data'] = se.values
transformed_tweet_data.insert(1,'profane_class',1)
print(transformed_tweet_data.count())

In [ ]:
# se = pd.Series(transformed_data_comment)
# transformed_comment_data['text_data'] = se.values
# transformed_comment_data.insert(1,'profane_class',1)
# print(transformed_comment_data.count())

In [ ]:
# transformed_tweet_data.head(10)
writer = pd.ExcelWriter('C:/Users/saath/Documents/DAEN 690/datasets/tweet_translated.xlsx')
# write dataframe to excel
transformed_tweet_data.to_excel(writer)
# save the excel
writer.save()
print('DataFrame is written successfully to Excel File.')

In [ ]:
# # transformed_tweet_data.head(10)
# writer = pd.ExcelWriter('C:/Users/saath/Documents/DAEN 690/datasets/comment_translated.xlsx')
# # write dataframe to excel
# transformed_comment_data.to_excel(writer)
# # save the excel
# writer.save()
# print('DataFrame is written successfully to Excel File.')